In [1]:
import multiprocessing as mp
import numpy as np
import dask.array as da
import pandas as pd
pd.set_option("display.max_columns",None)
path="C:/Users/HUGO/Desktop/Maestria en CIENCIA DE DATOS UANL/DatosMasivos/"

# Practica 1: Paralelizacion.

Parte 1: Investigar sobre las lazy variables y como se usa el Delay para problemas mas grandes.

Parte 2: Generen varias funciones y construyan un grafo de paralelización con al menos 4 cuellos de botella.


Extra challenge: Buscar ensemble learning en Scikit learn, utilizar minimo 3 algoritmos de machine learning para crear un modelo paralelo basado en un ensamble. Generar uno con dask y otro con la libreria de ensemble de sklearn, y comparar resultados. 

## Parte 1:
**Lazy variables y como se usa el Delay para problemas mas grandes**

Las lazy variables (también conocidas como variables perezosas o lazy properties) son variables en programación que se calculan o inicializan solo cuando se acceden por primera vez, en lugar de hacerlo de manera inmediata al crear el objeto que las contiene.

En otros términos, una variable perezosa es una propiedad que no se calcula hasta que se solicita, lo que puede ser útil cuando se trabaja con objetos complejos o costosos de crear. En lugar de crear todos los componentes de un objeto en el momento de su creación, se retrasa la creación de algunos de ellos hasta que realmente se necesiten, lo que puede mejorar el rendimiento y la eficiencia del programa.

El uso de variables perezosas puede ser especialmente beneficioso en lenguajes de programación que son interpretados o tienen un alto costo de inicialización, ya que permite diferir el costo de la inicialización hasta que sea realmente necesario.

**Delay:**

En algunos casos, puede ser beneficioso retrasar la inicialización de una variable aún más, especialmente en problemas más grandes que requieren más tiempo de cómputo o memoria.

Una forma de retrasar aún más la inicialización es utilizando la función Delay en lugar de una variable perezosa convencional. La función Delay es una técnica de programación que permite posponer la evaluación de una expresión hasta que se solicite su resultado.

Por ejemplo, supongamos que se tiene una función que realiza un cálculo costoso o tardado, pero que este cálculo solo se necesita en ciertas situaciones. En lugar de calcular el resultado de la función de inmediato, se puede utilizar la función Delay para posponer la evaluación hasta que sea necesaria. De esta forma, se puede ahorrar tiempo y memoria en el procesamiento de la aplicación.

## Parte 2:
**Generen varias funciones y construyan un grafo de paralelización con al menos 4 cuellos de botella.**

In [2]:
import dask.delayed as delayed
from dask.diagnostics import ProgressBar
import time
import random


In [7]:
lista=[random.randint(0, 9) for _ in range(10)]
def suma(x):
    return x + random.randint(0,9)

def resta(x):
    return x - random.randint(0,9)

def multiply_add(x,y):
    return(x*y)



In [ ]:
#Capa 1 -cuello 1
step1 = [delayed(suma)(i) for i in lista]
tot1 = delayed(sum)(step1)
#Capa 2- cuello 2
step2 = [delayed(resta)(i, tot1) for k in lista]
tot2 = delayed(sum)(step2)
#Capa 3 - cuello 3
step3 = [delayed(multiply_add)(i, tot2) for k in lista]
tot3 = delayed(sum)(step3)
#Capa 4 -cuello 4
step1 = [delayed(suma)(i,tot3) for i in lista]
tot4 = delayed(sum)(step4)
tot4.visualize()

In [2]:
path="C:/Users/HUGO/Desktop/Maestria en CIENCIA DE DATOS UANL/Aprendizaje automatico/"
loans=pd.read_csv(path+"loans_dummies.csv")
loans2=pd.read_csv(path+"loansmodified3.csv")

In [3]:
import dask.dataframe as dd
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import time


In [4]:
loans["paid"]=loans2["paid"]


In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#import dask_ml.joblib
from time import time

X=loans.drop("paid",axis=1)
y=loans.paid

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear los clasificadores individuales
logreg = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC()

# Crear el modelo de ensamble utilizando VotingClassifier de Scikit-learn
ensemble_sklearn = VotingClassifier(
    estimators=[('lr', logreg), ('rf', rf), ('svm', svm)],
    voting='hard'
)

# Medir el tiempo de ejecución del modelo de ensamble de Scikit-learn
start_time_sklearn = time()
ensemble_sklearn.fit(X_train, y_train)
end_time_sklearn = time()

# Calcular el tiempo de ejecución del modelo de ensamble de Scikit-learn
execution_time_sklearn = end_time_sklearn - start_time_sklearn

# Predecir utilizando el modelo de ensamble de Scikit-learn
y_pred_sklearn = ensemble_sklearn.predict(X_test)

# Calcular la precisión del modelo de ensamble de Scikit-learn
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)

# Crear el modelo de ensamble utilizando VotingClassifier de Dask-ML
ensemble_dask = VotingClassifier(
    estimators=[('lr', logreg), ('rf', rf), ('svm', svm)],
    voting='hard'
)

# Medir el tiempo de ejecución del modelo de ensamble de Dask-ML
start_time_dask = time()
with dask_ml.joblib.parallel_backend('dask'):
    ensemble_dask.fit(X_train, y_train)
end_time_dask = time()

# Calcular el tiempo de ejecución del modelo de ensamble de Dask-ML
execution_time_dask = end_time_dask - start_time_dask

# Predecir utilizando el modelo de ensamble de Dask-ML
y_pred_dask = ensemble_dask.predict(X_test)

# Calcular la precisión del modelo de ensamble de Dask-ML
accuracy_dask = accuracy_score(y_test, y_pred_dask)

# Comparar los tiempos de ejecución
print("Tiempo de ejecución del modelo de ensamble con Scikit-learn:", execution_time_sklearn)
print("Tiempo de ejecución del modelo de ensamble con Dask-ML:", execution_time_dask)
print("Precisión del modelo de ensamble con Scikit-learn:", accuracy_sklearn)
print("Precisión del modelo de ensamble con Dask-ML:", accuracy_dask)


C:\Users\HUGO\anaconda\envs\project2\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
